In [1]:
import pandas as pd
import numpy as np

path = '/home/platform/project/mrnadesign_platform/mrnadesign_api/workspace/mrnadesign_database/'

In [2]:
df1 = pd.read_csv(path + 'Antigen_se.csv')
print(df1.shape)
df2 = pd.read_csv(path + 'TAntigendb_se.csv')
print(df2.shape)
df3 = pd.read_csv(path + 'Tantigen_rna_seq.csv')
print(df3.shape)
df4 = pd.read_csv(path + 'Complete_3_UTR_ARE_Data/Complete_3_UTR_Data.csv')
print(df4.shape)

(505, 19)
(4295, 13)
(4306, 5)
(4884, 10)


In [21]:
merged_df = pd.merge(df2, df3, how='outer', on=['AgACC'])

merged_df['Antigen Name_x'] = merged_df['Antigen Name_x'].combine_first(merged_df['Antigen Name_y'])
merged_df = merged_df.drop(columns=['Antigen Name_y'])
merged_df.rename(columns={'Antigen Name_x': 'Antigen Name'}, inplace=True)
# merged_df = merged_df.to_numpy()

In [26]:
merged_df.shape

(4307, 16)

In [6]:
ddf1 = pd.DataFrame([
    ['a', 1, 11],
    ['b', 2, 22],
    ['c', 3, 33]
], columns=['letter', 'n1', 'n2'])
ddf1

,letter,n1,n2
0,a,1,11
1,b,2,22
2,c,3,33


In [7]:
ddf2 = pd.DataFrame([
    ['a', 111, 1111],
    ['c', 333, 3333],
    ['d', 444, 4444]
], columns=['letter', 'n1', 'n4'])
ddf2

,letter,n1,n4
0,a,111,1111
1,c,333,3333
2,d,444,4444


In [17]:
merged_df = pd.merge(ddf1, ddf2, how='outer', on=['letter'])
merged_df 

,letter,n1_x,n2,n1_y,n4
0,a,1.0,11.0,111.0,1111.0
1,b,2.0,22.0,NaN,NaN
2,c,3.0,33.0,333.0,3333.0
3,d,NaN,NaN,444.0,4444.0


In [18]:
merged_df['n1_x'] = merged_df['n1_x'].combine_first(merged_df['n1_y'])
merged_df = merged_df.drop(columns=['n1_y'])
merged_df.rename(columns={'n1_x': 'n1'}, inplace=True)
merged_df

,letter,n1,n2,n4
0,a,1.0,11.0,1111.0
1,b,2.0,22.0,NaN
2,c,3.0,33.0,3333.0
3,d,444.0,NaN,4444.0
